In [3]:
import ast
import pandas as pd
test_file = pd.read_csv('vt_merged.csv').drop('Unnamed: 0', axis = 1).iloc[15:18]
print(test_file)

from huggingface_hub import login
import os
from dotenv import load_dotenv

hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

from transformers import pipeline
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

model_id = "google/gemma-3-4b-it"

pipe = pipeline(
    "image-text-to-text",
    model=model_id,
    device=device,          
    torch_dtype=torch.bfloat16
)

# Testing ads + description

advertisement_examples = [
    "Billboard with product name and price",
    "Social media post promoting a sale",
    "Banner showing a company logo with a slogan",
    "Flyer with a discount coupon"
]

for idx, row in test_file.iterrows():
    image_list = ast.literal_eval(row['pics_collapsed'])  # convert string to list

    if not image_list:  # skip empty lists
        test_file.at[idx, "is_image_ad"] = ""
        test_file.at[idx, "image_keywords"] = ""
        continue

    answers = []
    descriptions = []

    for image_url in image_list:
        messages = [
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": (
                        "You are an AI assistant that classifies images as advertisements. "
                        "Always respond in two clearly labeled sections: Answer, Description."
                    )}
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "url": image_url},
                    {"type": "text", "text": "Is this image an advertisement?"}
                ]
            }
        ]

        output = pipe(messages, max_new_tokens=128)
        response = output[0]["generated_text"]

        answer, rationale = "No", ""
        for line in response.splitlines():
            if line.startswith("Answer:"):
                answer = line.split(":", 1)[1].strip()
            elif line.startswith("Description:"):
                rationale = line.split(":", 1)[1].strip()

        answers.append(answer)
        descriptions.append(rationale)

    # Final decision
    final_answer = "Yes" if "Yes" in answers else "No"
    test_file.at[idx, "is_image_ad"] = final_answer
    test_file.at[idx, "image_Keywords"] = " | ".join(descriptions)

    review_id                user_id           time  rating  \
15         15  113658698063006557094  1520354731256       5   
16         16  103134655243955314411  1628737744108       5   
17         17  101856865551768948430  1606162783440       1   

                                                 text  \
15                                                NaN   
16  Very honest and helpful. Glad I went there. I ...   
17  I had them replace two broken studs on my righ...   

                                       pics_collapsed  \
15                                                 []   
16                                                 []   
17  ['https://lh5.googleusercontent.com/p/AF1QipPQ...   

                                       resp_collapsed          name  \
15                                                NaN  Carr's Gifts   
16  Wow, Angela, we love positive feedback! Thank ...         Midas   
17  Hi Henry our Customer Service Team would be ha...         Midas   

   d

Using device: mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps
/var/folders/sd/m0m5n5_52c7_yt7wl56_jyjm0000gn/T/ipykernel_19945/76455890.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_file.at[idx, "is_image_ad"] = ""
/var/folders/sd/m0m5n5_52c7_yt7wl56_jyjm0000gn/T/ipykernel_19945/76455890.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_file.at[idx, "image_keywords"] = ""


AttributeError: 'list' object has no attribute 'splitlines'

In [5]:
import torch

torch.mps.empty_cache()